In [ ]:
#XOR
import numpy as np
from matplotlib import pyplot as plt
def sigmoid(z):
return 1 / (1 + np.exp(-z))
def initialize_params(input_features, hidden_neurons, output_features):
W1 = np.random.randn(hidden_neurons, input_features)
W2 = np.random.randn(output_features, hidden_neurons)
b1 = np.zeros((hidden_neurons, 1))
b2 = np.zeros((output_features, 1))
return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
def forward_propagation(X, Y, params):
m = X.shape[1]
Z1 = np.dot(params["W1"], X) + params["b1"]
A1 = sigmoid(Z1)
Z2 = np.dot(params["W2"], A1) + params["b2"]
A2 = sigmoid(Z2)
logprobs = np.multiply(np.log(A2), Y) + np.multiply(np.log(1 - A2), (1 - Y))
cost = -np.sum(logprobs) / m
return cost, (Z1, A1, params["W1"], params["b1"], Z2, A2, params["W2"], params["b2"]),
A2
def backward_propagation(X, Y, cache):
m = X.shape[1]
Z1, A1, W1, b1, Z2, A2, W2, b2 = cache
dZ2 = A2 - Y
dW2 = np.dot(dZ2, A1.T) / m
db2 = np.sum(dZ2, axis=1, keepdims=True)
dA1 = np.dot(W2.T, dZ2)
dZ1 = np.multiply(dA1, A1 * (1 - A1))
dW1 = np.dot(dZ1, X.T) / m
db1 = np.sum(dZ1, axis=1, keepdims=True) / m
return {"dZ2": dZ2, "dW2": dW2, "db2": db2, "dZ1": dZ1, "dW1": dW1, "db1": db1}
def update_params(params, gradients, learning_rate):
params["W1"] -= learning_rate * gradients["dW1"]
params["W2"] -= learning_rate * gradients["dW2"]
params["b1"] -= learning_rate * gradients["db1"]
params["b2"] -= learning_rate * gradients["db2"]
return params
X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]]) # XOR input
Y = np.array([[0, 1, 1, 0]]) # XOR output
hidden_neurons, input_features, output_features = 2, X.shape[0], Y.shape[0]
params = initialize_params(input_features, hidden_neurons, output_features)
epochs, learning_rate = 100000, 0.01
losses = np.zeros((epochs, 1))
for i in range(epochs):
losses[i, 0], cache, A2 = forward_propagation(X, Y, params)
gradients = backward_propagation(X, Y, cache)
params = update_params(params, gradients, learning_rate)
plt.plot(losses)
plt.xlabel("EPOCHS")
plt.ylabel("Loss value")
plt.show()
X = np.array([[1, 1, 0, 0], [0, 1, 0, 1]]) # XOR input
cost, _, A2 = forward_propagation(X, Y, params)
prediction = (A2 > 0.5) * 1.0
print(prediction)

In [ ]:
#Digit Recognition
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = keras.Sequential([
keras.layers.Flatten(input_shape=(28, 28)),
keras.layers.Dense(128, activation='relu'),
keras.layers.Dropout(0.2),
keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')
predictions = model.predict(x_test)
num_images_to_display = 5
plt.figure(figsize=(15, 5))
for i in range(num_images_to_display):
plt.subplot(1, num_images_to_display, i + 1)
plt.imshow(x_test[i], cmap='gray')
plt.title(f"Predicted: {np.argmax(predictions[i])}\nActual: {y_test[i]}")
plt.axis('off')
plt.show()

In [ ]:
#XRay images
import logging
import os
import shutil
import sys
import tempfile
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
from skimage.util import random_noise
from tqdm import trange
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import CacheDataset, DataLoader
from monai.networks.nets import AutoEncoder
from monai.transforms import (
EnsureChannelFirstD,
Compose,
LoadImageD,
RandFlipD,
RandRotateD,
RandZoomD,
ScaleIntensityD,
EnsureTypeD,
Lambda,
)
from monai.utils import set_determinism
print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
set_determinism(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def plot_ims(ims, shape=None, figsize=(10, 10), titles=None):
shape = (1, len(ims)) if shape is None else shape
plt.subplots(*shape, figsize=figsize)
for i, im in enumerate(ims):
plt.subplot(*shape, i + 1)
im = plt.imread(im) if isinstance(im, str) else torch.squeeze(im)
plt.imshow(im, cmap="gray")
if titles is not None:
plt.title(titles[i])
plt.axis("off")
plt.tight_layout()
plt.show()
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)
resource = "https://github.com/Project-MONAI/MONAI-extra-test
data/releases/download/0.8.1/MedNIST.tar.gz"
md5 = "0bc7306e7427e00ad1c5526a6677552d"
compressed_file = os.path.join(root_dir, "MedNIST.tar.gz")
data_dir = os.path.join(root_dir, "MedNIST")
if not os.path.exists(data_dir):
download_and_extract(resource, compressed_file, root_dir, md5)
scan_type = "Hand"
im_dir = os.path.join(data_dir, scan_type)
all_filenames = [os.path.join(im_dir, filename) for filename in os.listdir(im_dir)]
random.shuffle(all_filenames)
rand_images = np.random.choice(all_filenames, 8, replace=False)
plot_ims(rand_images, shape=(2, 4))
test_frac = 0.2
num_test = int(len(all_filenames) * test_frac)
num_train = len(all_filenames) - num_test
train_datadict = [{"im": fname} for fname in all_filenames[:num_train]]
test_datadict = [{"im": fname} for fname in all_filenames[-num_test:]]
NoiseLambda = Lambda(
lambda d: {
"orig": d["im"],
"gaus": torch.tensor(random_noise(d["im"], mode="gaussian"), dtype=torch.float32),
"s&p": torch.tensor(random_noise(d["im"], mode="s&p", salt_vs_pepper=0.1)),
}
)
train_transforms = Compose(
[
LoadImageD(keys=["im"]),
EnsureChannelFirstD(keys=["im"]),
ScaleIntensityD(keys=["im"]),
RandRotateD(keys=["im"], range_x=np.pi / 12, prob=0.5, keep_size=True),
RandFlipD(keys=["im"], spatial_axis=0, prob=0.5),
RandZoomD(keys=["im"], min_zoom=0.9, max_zoom=1.1, prob=0.5),
EnsureTypeD(keys=["im"]),
NoiseLambda,
]
)
test_transforms = Compose(
[
LoadImageD(keys=["im"]),
EnsureChannelFirstD(keys=["im"]),
ScaleIntensityD(keys=["im"]),
EnsureTypeD(keys=["im"]),
NoiseLambda,
]
)
batch_size = 300
num_workers = 10
train_ds = CacheDataset(train_datadict, train_transforms, num_workers=num_workers)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
num_workers=num_workers)
test_ds = CacheDataset(test_datadict, test_transforms, num_workers=num_workers)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True,
num_workers=num_workers)
def get_single_im(ds):
loader = torch.utils.data.DataLoader(ds, batch_size=1, num_workers=10, shuffle=True)
itera = iter(loader)
return next(itera)
data = get_single_im(train_ds)
plot_ims([data["orig"], data["gaus"], data["s&p"]], titles=["orig", "Gaussian", "s&p"])
def train(dict_key_for_training, max_epochs=10, learning_rate=1e-3):
model = AutoEncoder(
spatial_dims=2,
in_channels=1,
out_channels=1,
channels=(4, 8, 16, 32),
strides=(2, 2, 2, 2),
).to(device)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
epoch_loss_values = []
t = trange(max_epochs, desc=f"{dict_key_for_training} -- epoch 0, avg loss: inf",
leave=True)
for epoch in t:
model.train()
epoch_loss = 0
step = 0
for batch_data in train_loader:
step += 1
inputs = batch_data[dict_key_for_training].to(device)
optimizer.zero_grad()
outputs = model(inputs)
loss = loss_function(outputs, batch_data["orig"].to(device))
loss.backward()
optimizer.step()
epoch_loss += loss.item()
epoch_loss /= step
epoch_loss_values.append(epoch_loss)
t.set_description(f"{dict_key_for_training} -- epoch {epoch + 1}" + f", average loss:
{epoch_loss:.4f}")
return model, epoch_loss_values
max_epochs = 50
training_types = ["orig", "gaus", "s&p"]
models = []
epoch_losses = []
for training_type in training_types:
model, epoch_loss = train(training_type, max_epochs=max_epochs)
models.append(model)
epoch_losses.append(epoch_loss)
plt.figure()
plt.title("Epoch Average Loss")
plt.xlabel("epoch")
for y, label in zip(epoch_losses, training_types):
x = list(range(1, len(y) + 1))
(line,) = plt.plot(x, y)
line.set_label(label)
plt.legend()
data = get_single_im(test_ds)
recons = []
for model, training_type in zip(models, training_types):
im = data[training_type]
recon = model(im.to(device)).detach().cpu()
recons.append(recon)
plot_ims(
[data["orig"], data["gaus"], data["s&p"]] + recons,
titles=["orig", "Gaussian", "S&P"] + ["recon w/\n" + x for x in training_types],
shape=(2, len(training_types)),
)
def inference(dict_key_for_training, trained_model):
model = trained_model
model.eval()
im = data[dict_key_for_training]
recon = model(im.to(device)).detach().cpu()
return recon
reconstructions = []
for trained_model, training_type in zip(models, training_types):
recon = inference(training_type, trained_model)
reconstructions.append(recon)
plot_ims(
[data["orig"], data["gaus"], data["s&p"]] + reconstructions,
titles=["orig", "Gaussian", "S&P"] + ["recon w/\n" + x for x in training_types],
shape=(2, len(training_types)),
)

In [ ]:
#Speech Recognition
import speech_recognition as sr
def speech_recognition_example():
recognizer = sr.Recognizer()
with sr.Microphone() as source:
print("Say something:")
recognizer.adjust_for_ambient_noise(source)
audio = recognizer.listen(source, timeout=5)
try:
text = recognizer.recognize_google(audio)
print(f"You said: {text}")
except sr.UnknownValueError:
print("Speech recognition could not understand audio")
except sr.RequestError as e:
print(f"Could not request results from Google Web Speech API; {e}")
if __name__ == "__main__":
speech_recognition_example()

In [ ]:
#Traffic Analysis
import cv2
import csv
import math
import numpy as np
from tracker import EuclideanDistTracker
cap = cv2.VideoCapture("bridge.mp4")
input_size = 320
tracker = EuclideanDistTracker()
confThreshold = 0.2
nmsThreshold = 0.2
font_color = (0, 0, 255)
font_size = 0.5
font_thickness = 2
middle_line_position = 225
up_line_position = middle_line_position - 15
down_line_position = middle_line_position + 15
classesFile = "coco.names"
classNames = open(classesFile).read().strip().split('\n')
print(classNames)
print(len(classNames))
required_class_index = [2, 3, 5, 7]
detected_classNames = []
modelConfiguration = 'yolov3-320.cfg'
modelWeights = 'yolov3-320.weights'
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')
def find_center(x, y, w, h):
x1 = int(w / 2)
y1 = int(h / 2)
cx = x + x1
cy = y + y1
return cx, cy
temp_up_list = []
temp_down_list = []
up_list = [0, 0, 0, 0]
down_list = [0, 0, 0, 0]
def count_vehicle(box_id, img):
x, y, w, h, id, index = box_id
center = find_center(x, y, w, h)
ix, iy = center
if (iy > up_line_position) and (iy < middle_line_position):
if id not in temp_up_list:
temp_up_list.append(id)
elif iy < down_line_position and iy > middle_line_position:
if id not in temp_down_list:
temp_down_list.append(id)
elif iy < up_line_position:
if id in temp_down_list:
temp_down_list.remove(id)
up_list[index] = up_list[index] + 1
elif iy > down_line_position:
if id in temp_up_list:
temp_up_list.remove(id)
down_list[index] = down_list[index] + 1
cv2.circle(img, center, 2, (0, 0, 255), -1)
def postProcess(outputs, img):
global detected_classNames
height, width = img.shape[:2]
211720243052
boxes = []
classIds = []
confidence_scores = []
detection = []
for output in outputs:
for det in output:
scores = det[5:]
classId = np.argmax(scores)
confidence = scores[classId]
if classId in required_class_index:
if confidence > confThreshold:
w, h = int(det[2] * width), int(det[3] * height)
x, y = int((det[0] * width) - w / 2), int((det[1] * height) - h / 2)
boxes.append([x, y, w, h])
classIds.append(classId)
confidence_scores.append(float(confidence)
indices = cv2.dnn.NMSBoxes(boxes, confidence_scores, confThreshold, nmsThreshold)
for i in indices.flatten():
x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
color = [int(c) for c in colors[classIds[i]]
name = classNames[classIds[i]]
detected_classNames.append(name)
cv2.putText(img, f'{name.upper()} {int(confidence_scores[i] * 100)}%',
(x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
detection.append([x, y, w, h, required_class_index.index(classIds[i])])
boxes_ids = tracker.update(detection)
for box_id in boxes_ids:
count_vehicle(box_id, img)
def realTime():
while True:
success, img = cap.read()
img = cv2.resize(img, (0, 0), None, 0.5, 0.5)
ih, iw, channels = img.shape
blob = cv2.dnn.blobFromImage(img, 1 / 255, (input_size, input_size), [0, 0, 0], 1,
crop=False)
net.setInput(blob)
layersNames = net.getLayerNames()
outputNames = [(layersNames[i - 1]) for i in net.getUnconnectedOutLayers()]
outputs = net.forward(outputNames)
postProcess(outputs, img)
cv2.line(img, (0, middle_line_position), (iw, middle_line_position), (255, 0, 255), 2)
cv2.line(img, (0, up_line_position), (iw, up_line_position), (0, 0, 255), 2)
cv2.line(img, (0, down_line_position), (iw, down_line_position), (0, 0, 255), 2)
cv2.putText(img, "Up", (110, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size,
font_color, font_thickness)
cv2.putText(img, "Down", (160, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size,
font_color, font_thickness)
cv2.putText(img, "Car: " + str(up_list[0]) + " " + str(down_list[0]), (20, 40),
cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
cv2.putText(img, "Motorbike: " + str(up_list[1]) + " " + str(down_list[1]), (20, 60),
cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
cv2.putText(img, "Bus: " + str(up_list[2]) + " " + str(down_list[2]), (20, 80),
cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
cv2.putText(img, "Truck: " + str(up_list[3]) + " " + str(down_list[3]), (20, 100),
cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
cv2.imshow('Output', img)
if cv2.waitKey(1) == ord('q'):
break
with open("data.csv", 'w') as f1:
cwriter = csv.writer(f1)
cwriter.writerow(['Direction', 'car', 'motorbike', 'bus', 'truck'])
up_list.insert(0, "Up")
down_list.insert(0, "Down")
cwriter.writerow(up_list)
cwriter.writerow(down_list)
print("Data saved at 'data.csv'")
f1.close()
cap.release()
cv2.destroyAllWindows()
if __name__ == '__main__':
realTime()

In [ ]:
#Online fraud detect
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
dataset = pd.read_csv('share_market_data.csv')
X = dataset.drop('Target', axis=1)
y = dataset['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
model = IsolationForest(contamination=0.1, random_state=1)
model.fit(X_train)
y_pred = model.predict(X_test)
y_pred = [1 if x == -1 else 0 for x in y_pred]
print(classification_report(y_test, y_pred))

In [ ]:
#Image Augmentation
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
import os
import numpy as np
from PIL import Image
datagen = ImageDataGenerator(
rotation_range=45,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='constant', cval=125)
x = io.imread('msd_abd/msd/msd.jpg')
x = x.reshape((1, ) + x.shape)
i = 0
for batch in datagen.flow(x, batch_size=16, save_to_dir='augmented', save_prefix='aug',
save_format='png'):
i += 1
if i > 20:
break
SIZE = 128
dataset = []
image_directory = 'test_folder/'
my_images = os.listdir(image_directory)
for i, image_name in enumerate(my_images):
if image_name.split('.')[1] == 'jpg':
image = io.imread(image_directory + image_name)
image = Image.fromarray(image, 'RGB')
image = image.resize((SIZE, SIZE))
dataset.append(np.array(image))
x = np.array(dataset)
i = 0
for batch in datagen.flow_from_directory(
directory='msd_abd/',
batch_size=16,
target_size=(256, 256),
color_mode="rgb",
save_to_dir='augmented',
save_prefix='aug',
save_format='png'):
i += 1
if i > 31:
break

In [ ]:
#Sentiment Analysis
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
data = pd.read_csv('../input/Sentiment.csv')
data = data[['text', 'sentiment']]
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1])
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
batch_size = 32
model.fit(X_train, Y_train, epochs=7, batch_size=batch_size, verbose=2)
validation_size = 1500
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
result = model.predict(X_validate[x].reshape(1, X_test.shape[1]), batch_size=1,
verbose=2)[0]
if np.argmax(result) == np.argmax(Y_validate[x]):
if np.argmax(Y_validate[x]) == 0:
neg_correct += 1
else:
pos_correct += 1
if np.argmax(Y_validate[x]) == 0:
neg_cnt += 1
else:
pos_cnt += 1
twt = ['Meetings: Because none of us is as dumb as all of us.']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
sentiment = model.predict(twt, batch_size=1, verbose=2)[0]
if np.argmax(sentiment) == 0:
print("negative")
elif np.argmax(sentiment) == 1:
print("positive")